In this notebook, me and my team, we are going to scrape an ecommerce website. We'll get into each individual product page and retrieve our information from there. This is the website we are going to scrape <a href="https://www.amazon.com/s?i=mobile&rh=n%3A2335752011%2Cp_72%3A2491149011&pd_rd_r=30305d08-93c6-4f70-aca0-5bdcd0619f0a&pd_rd_w=V3rlu&pd_rd_wg=cnq1i&pf_rd_p=f5c158e1-98f7-4998-94b8-d7306c066086&pf_rd_r=Q0RJ0DQSNB5EM8EWZX6G&qid=1649783074&ref=sr_pg_1">Amazon</a>  – it's an French online shop.

In [39]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import ast


Maintenant, nous allons définir l'URL de base de la page principale car nous en aurons besoin lorsque nous construirons nos URL pour chacun des produits individuels.

De plus, nous enverrons un agent utilisateur sur chaque requête HTTP, car si nous avons effectuez une requête GET à l'aide de requests , l'agent utilisateur est Python par défaut, ce qui peut être bloqué.

Donc, pour remplacer cela, nous allons déclarer une variable qui stockera notre user-agent.

In [2]:
from selectorlib import Extractor
import requests 
import json 
from time import sleep

In [16]:
import pandas as pd
import numpy as np
from ast import literal_eval

data=open("D:\INPT2\Data mining\DataMining-Project\web Scraping\output_1.json","r")
data=[literal_eval(i) for i in data]
print(len(data))
df = pd.DataFrame.from_dict(data, orient='columns')
df=df.replace("null", np.NaN)

1158


In [17]:
list(df.columns)

['name',
 'price',
 'images',
 'rating',
 'product_description',
 'number_of_reviews']

In [5]:
df.isnull().sum()

name                   926
price                  977
images                 926
rating                 946
product_description    936
number_of_reviews      946
dtype: int64

In [19]:
df=df.dropna(axis=0, how="all")
df=df.drop_duplicates()

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107 entries, 13 to 1151
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name                 107 non-null    object
 1   price                57 non-null     object
 2   images               107 non-null    object
 3   rating               95 non-null     object
 4   product_description  101 non-null    object
 5   number_of_reviews    95 non-null     object
dtypes: object(6)
memory usage: 5.9+ KB


In [21]:
df.to_csv("D:\INPT2\Data mining\DataMining-Project\web Scraping\Data_Amazon.csv", mode='a', header=False)

In [22]:
df=pd.read_csv("D:\INPT2\Data mining\DataMining-Project\web Scraping\Data_Amazon.csv")


### transform product's images link to list

In [40]:
df["images"]=df["images"].apply(lambda x: list(ast.literal_eval(x).keys()))

### Cleaning Name and Product_Description

In [50]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

from nltk.stem.porter import PorterStemmer

In [51]:
def cleaning(text):
    # split into words
    tokens = word_tokenize(text)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    #Stemming Words
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in words]

    return stemmed

apply function cleaning in name and product_description

In [61]:
df["name_clean"]=df["name"].apply(lambda x: cleaning(x))
df["category_clean"]=df["product_description"].apply(lambda x: cleaning(x))

TypeError: expected string or bytes-like object

In [60]:
df[["name","name_clean"]].head(5)

,name,name_clean
0,"Dirt Devil Simpli-Stik Vacuum Cleaner, 3-in-1 ...","[dirt, devil, simplistik, vacuum, cleaner, han..."
1,Tineco Floor ONE S3 Cordless Hardwood Floors C...,"[tineco, floor, one, cordless, hardwood, floor..."
2,"Drop PLNT HNGR 4FT 3/8""12","[drop, plnt, hngr]"
3,"Cuisinart Waffle Maker, Double Belgian, Stainl...","[cuisinart, waffl, maker, doubl, belgian, stai..."
4,Waterdrop DA29-00020B Refrigerator Water Filte...,"[waterdrop, refriger, water, filter, replac, s..."


# -----------------